In [1]:
import numpy as np
import pandas as pd
import julian
import datetime
import os
from scipy.io import savemat, loadmat
import matplotlib.pyplot as plt
import sunpy.map
import matplotlib.dates as mdates
%matplotlib inline

import gzip

import astropy.constants as const
import astropy.units as u
from astropy.coordinates import SkyCoord
from sunpy.net import attrs as attrs

import sunpy.coordinates.frames as frames
import pfsspy
from pfsspy import tracing

import sunpy.data.sample
from sunpy.net import Fido
import warnings
from two_step_ballistic_backmapping_method import *
import requests
import sunpy.map
import matplotlib.pyplot as plt
from sunpy.net import Fido, attrs as a
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, urlretrieve
import cgi

In [2]:
def get_mode(data):
    return np.nanmean(data)
def Carrington_To_Helioprojective(carr_lon, carr_lat, obstime, observer):
    # 将卡林顿坐标转换到天空平面，比如转到AIA图中x,y,单位角秒。
    Carrington_lon_lat = SkyCoord(lon=carr_lon * u.deg, lat=carr_lat * u.deg, \
                                  frame=frames.HeliographicCarrington, obstime=obstime, observer=observer)
    helioprojective = Carrington_lon_lat.transform_to(frames.Helioprojective)
    x_arcsec = helioprojective.Tx.value
    y_arcsec = helioprojective.Ty.value
    return x_arcsec, y_arcsec
def get_window_mode_time_series(time_series,data_series,window_length):
    '''
    data_series, 1D time series
    window_length, scalar, 12 hours
    '''
    length_of_data = np.shape(data_series)[0]
    data_index_arr = np.array(range(length_of_data))
    mode_velocity_arr = np.array([])
    for i in time_series:
        data_index_in_window = (time_series > i-window_length/2) & (time_series < i+window_length/2)
        mode_velocity = get_mode(data_series[data_index_in_window])
        mode_velocity_arr = np.append(mode_velocity_arr,mode_velocity)
    return mode_velocity_arr


def get_url_paths(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    return parent

def download_solo_eui_map(obs_datetime, local_data_dir='./'):
    year_str = '{:04d}'.format(obs_datetime.year)
    month_str = '{:02d}'.format(obs_datetime.month)
    day_str = '{:02d}'.format(obs_datetime.day)
    hour_str = '{:02d}'.format(obs_datetime.hour)
    minute_str = '{:02d}'.format(obs_datetime.minute)
    second_str = '{:02d}'.format(obs_datetime.second)
    
    download_file_seconds = obs_datetime.hour*3600+obs_datetime.minute*60+obs_datetime.second
    # 定义URL和文件名
    _url = "https://www.sidc.be/EUI/data/L2/"+year_str +"/"+month_str +"/"+day_str +"/"
    _file = "solo_L2_eui-fsi174-image_"+year_str+month_str+day_str+"T"
    # 下载FITS文件 到 本地数据文件夹
    try:
        file_list = get_url_paths(_url,'.fits')
        if len(file_list)>0:
            second_num_arr = np.array([])
            idx_arr = np.array([])
            for idx in range(len(file_list)):
                if _file in file_list[idx]:
                    file_timestamp = file_list[idx].split(_file)[1]
                    second_num = int(file_timestamp[0:2])*3600 + int(file_timestamp[2:4])*60 + int(file_timestamp[4:6])
                    second_num_arr = np.append(second_num_arr,second_num)
                    idx_arr = np.append(idx_arr,idx)
            if idx_arr.size>0:
                idx = int(idx_arr[np.argmin(np.abs(second_num_arr-download_file_seconds))])
                filename_need_to_downloaded = file_list[idx]
                if filename_need_to_downloaded and (np.abs(int(filename_need_to_downloaded.split(_file)[1][0:2]) - obs_datetime.hour) <= 1):
                    download_filename = local_data_dir + filename_need_to_downloaded
                    if os.path.isfile(download_filename):
                        return download_filename
                    else:
                        response = requests.get(_url+filename_need_to_downloaded)
                        open(local_data_dir + filename_need_to_downloaded, "wb").write(response.content)
                        return download_filename
    except:
        pass

**做回溯**

In [ ]:
from spacepy import pycdf
data_dir = '/Users/chuanpenghou/research/work/SolO_WIND/data/'
data_file_name = 'output-wnd_swe_v_0_wnd_swe_vth_wnd_swe_n_wnd_bh_0_wnd_bmagh_solo_b_rtn_hr_tot_s_2023083235954000.cdf'

data_cdf = pycdf.CDF(data_dir+data_file_name)

print(data_cdf)
# load SolO data
Epoch_orbit = data_cdf['AMDA_TIME'][...]
lon_HCI = data_cdf['so_lon_hci_3575007927372174716'][...] # unit: degree
lat_HCI = data_cdf['so_lat_hci_3575007927372174716'][...] # unit: degree
dist_HCI = data_cdf['so_r_sun_3575007927372174716'][...] # unit: au
Vr_vect = data_cdf['pas_momgr1_v_rtn_4771868507688303223_1'][...]
N_vect = data_cdf['pas_momgr_n_4771868507688303223'][...]
T_vect = data_cdf['pas_momgr_tav_4771868507688303223'][...]

In [ ]:
dir_data_PFSS = '/Users/chuanpenghou/research/work/SolO_WIND/data/' # save GONG synoptic map. 存放Gong的磁图
dir_data = '/Users/chuanpenghou/research/work/SolO_WIND/data/' # save data generated by this code. 保存生成的数据

datetime_vect = pd.to_datetime(Epoch_orbit)
JulDay_vect = datetime_vect.to_julian_date()
JulDay_vect = JulDay_vect.to_numpy()
Vr_localmean = get_window_mode_time_series(JulDay_vect, Vr_vect, 4.0/24)

skycoord_HCI_vect = SkyCoord(lon=lon_HCI * u.deg, lat=lat_HCI * u.deg, distance=dist_HCI * u.AU, \
                             frame=frames.HeliocentricInertial, obstime=datetime_vect, observer="sun")
skycoord_HC_vect = skycoord_HCI_vect.transform_to(frames.HeliographicCarrington)

lon_HC_vect = skycoord_HC_vect.lon.degree  # unit: degree
lat_HC_vect = skycoord_HC_vect.lat.degree  # unit: degree
r_HC_vect = skycoord_HC_vect.radius  # unit: AU

JulDay_trace_beg = julian.to_jd(datetime.datetime(2023, 3, 25, 0, 0, 0))
JulDay_trace_end = julian.to_jd(datetime.datetime(2023, 4, 4, 0, 0, 0))

obstime_arr = np.array([])
JulDay_trace_arr = np.array([])
MFL_photosphere_lon_deg_arr = np.array([])
MFL_photosphere_lat_deg_arr = np.array([])
MFL_lon_arcsec_arr = np.array([])
MFL_lat_arcsec_arr = np.array([])
lon_footpoint_on_SourceSurface_deg_arr = np.array([])
lat_footpoint_on_SourceSurface_deg_arr = np.array([])
lon_beg_arr = np.array([])
lat_beg_arr = np.array([])

print(JulDay_trace_beg)
save_filename = 'from_' + str(julian.from_jd(JulDay_trace_beg)) + '_to_' + str(julian.from_jd(JulDay_trace_end))
print(save_filename)

for JulDay_index in range(JulDay_vect.size):
    if JulDay_vect[JulDay_index] > JulDay_trace_beg and JulDay_vect[JulDay_index] < JulDay_trace_end:
        JulDay_trace = JulDay_vect[JulDay_index]
        datetime_trace = julian.from_jd(JulDay_trace)
        print('datetime_trace: ', julian.from_jd(JulDay_trace))
        
        r_beg_au = r_HC_vect[JulDay_index].value
        lat_beg_deg = lat_HC_vect[JulDay_index]
        lon_beg_deg = lon_HC_vect[JulDay_index]
        Vr_beg_kmps = np.abs(Vr_localmean[JulDay_index])

        print('Vr_beg_kmps:', Vr_beg_kmps)

        if np.isnan(Vr_beg_kmps) or abs(Vr_beg_kmps) > 1e15:
            continue
        else:
            r_source_surfae_rs = 2.5 # Rs
            
            # two step backmapping
            r_footpoint_on_SourceSurface_rs, lon_footpoint_on_SourceSurface_deg, lat_footpoint_on_SourceSurface_deg, \
            MFL_photosphere_lon_deg, MFL_photosphere_lat_deg,_\
                =two_step_backmapping(datetime_trace, r_beg_au, lat_beg_deg, lon_beg_deg, Vr_beg_kmps, r_source_surfae_rs, dir_data_PFSS)
            print('beg_lon, beg_lat: ',lon_beg_deg,lat_beg_deg)
            print('end_lon, end_lat: ',MFL_photosphere_lon_deg,MFL_photosphere_lat_deg)
            #  transform to Helioprojective frame (arcsec)
            from_au_to_Rs = 214.9
            from_Rs_to_km = 6.96e5
            transit_time_unit_day = (r_beg_au * from_au_to_Rs - 1) * from_Rs_to_km / Vr_beg_kmps / 86400
            obstime = julian.from_jd(JulDay_trace - transit_time_unit_day)
            [MFL_lon_arcsec, MFL_lat_arcsec] = Carrington_To_Helioprojective(MFL_photosphere_lon_deg, MFL_photosphere_lat_deg, obstime, observer='earth')
            # MFL_lon_arcsec, MFL_lat_arcsec:是在地球处，看到的日面位置。
            
            # save all variables
            obstime_arr = np.append(obstime_arr,julian.to_jd(obstime))
            JulDay_trace_arr = np.append(JulDay_trace_arr,JulDay_trace)
            MFL_photosphere_lon_deg_arr = np.append(MFL_photosphere_lon_deg_arr,MFL_photosphere_lon_deg)
            MFL_photosphere_lat_deg_arr = np.append(MFL_photosphere_lat_deg_arr,MFL_photosphere_lat_deg)
            MFL_lon_arcsec_arr = np.append(MFL_lon_arcsec_arr,MFL_lon_arcsec)
            MFL_lat_arcsec_arr = np.append(MFL_lat_arcsec_arr,MFL_lat_arcsec)
            lon_footpoint_on_SourceSurface_deg_arr = np.append(lon_footpoint_on_SourceSurface_deg_arr,lon_footpoint_on_SourceSurface_deg)
            lat_footpoint_on_SourceSurface_deg_arr = np.append(lat_footpoint_on_SourceSurface_deg_arr,lat_footpoint_on_SourceSurface_deg)
            lon_beg_arr = np.append(lon_beg_arr,lon_beg_deg)
            lat_beg_arr = np.append(lat_beg_arr,lat_beg_deg)
            
            
mdict = {'obstime_arr':obstime_arr,'JulDay_trace_arr':JulDay_trace_arr,
        'MFL_photosphere_lon_deg_arr':MFL_photosphere_lon_deg_arr,
        'MFL_photosphere_lat_deg_arr':MFL_photosphere_lat_deg_arr,
        'MFL_lon_arcsec_arr':MFL_lon_arcsec_arr,
        'MFL_lat_arcsec_arr':MFL_lat_arcsec_arr,
        'lon_footpoint_on_SourceSurface_deg_arr':lon_footpoint_on_SourceSurface_deg_arr,
        'lat_footpoint_on_SourceSurface_deg_arr':lat_footpoint_on_SourceSurface_deg_arr,
        'lon_beg_arr':lon_beg_arr,
        'lat_beg_arr':lat_beg_arr}
filename = 'SolO_trace_back_'+save_filename
dir_and_filename = dir_data + filename
print(dir_and_filename)
savemat(dir_and_filename, mdict)
print('Trace Back Done!')


**读取回溯数据并画图**

In [3]:
from spacepy import pycdf
import matplotlib as mpl

#读取高分辨率SolO/PAS数据
data_dir = '/Users/chuanpenghou/research/work/SolO_WIND/data/'
data_file_name = 'output-pas_momgr1_v_rtn_0_wnd_swe_v_0_wnd_swe_vth_wnd_swe_n_wnd_bh_0_wnd_bmagh__2023083235954000.cdf'
data_cdf = pycdf.CDF(data_dir+data_file_name)
Epoch_orbit = data_cdf['AMDA_TIME'][...]
Vr_vect = data_cdf['pas_momgr1_v_rtn_1'][...]
N_vect = data_cdf['so_pas_momgr1_N_13319874537646668802'][...]
T_vect = data_cdf['so_pas_momgr1_T_13319874537646668802'][...]
Vr_vect[Vr_vect<250] = np.nan
N_vect[N_vect>100] = np.nan
T_vect[T_vect>80] = np.nan

#读取回溯数据
dir_and_filename = '/Users/chuanpenghou/research/work/SolO_WIND/data/SolO_trace_back_from_2023-03-25_00_to_2023-04-04_00'
print(dir_and_filename)
data = loadmat(dir_and_filename)
# mdict = {'obstime_arr':obstime_arr,'JulDay_trace_arr':JulDay_trace_arr,
#         'MFL_photosphere_lon_deg_arr':MFL_photosphere_lon_deg_arr,
#         'MFL_photosphere_lat_deg_arr':MFL_photosphere_lat_deg_arr,
#         'MFL_lon_arcsec_arr':MFL_lon_arcsec_arr,
#         'MFL_lat_arcsec_arr':MFL_lat_arcsec_arr,
#         'lon_footpoint_on_SourceSurface_deg_arr':lon_footpoint_on_SourceSurface_deg_arr,
#         'lat_footpoint_on_SourceSurface_deg_arr':lat_footpoint_on_SourceSurface_deg_arr,
#         'lon_beg_arr':lon_beg_arr,
#         'lat_beg_arr':lat_beg_arr}

/Users/chuanpenghou/research/work/SolO_WIND/data/SolO_trace_back_from_2023-03-25_00_to_2023-04-04_00


In [ ]:
imaging_obstime_julian_arr = data['obstime_arr'][-1]
JulDay_trace_arr = data['JulDay_trace_arr'][-1]
MFL_photosphere_lon_deg_arr = data['MFL_photosphere_lon_deg_arr'][-1]
MFL_photosphere_lat_deg_arr = data['MFL_photosphere_lat_deg_arr'][-1]
lon_beg_arr = data['lon_beg_arr'][-1]
lat_beg_arr = data['lat_beg_arr'][-1]
#图片文件夹
fulldisk_figure_dir = '/Users/chuanpenghou/research/work/SolO_WIND/figures/fulldisk/'
synoptic_figure_dir = '/Users/chuanpenghou/research/work/SolO_WIND/figures/synoptic/'
#本地eui数据文件夹
local_data_dir = '/Users/chuanpenghou/research/work/SolO_WIND/data/SolO_EUI/'

for imaging_obstime_julian, JulDay_trace, MFL_photosphere_lon_deg, MFL_photosphere_lat_deg, lon_beg, lat_beg \
    in zip(imaging_obstime_julian_arr, JulDay_trace_arr, MFL_photosphere_lon_deg_arr, MFL_photosphere_lat_deg_arr, lon_beg_arr, lat_beg_arr):
    
    datetime_trace = julian.from_jd(JulDay_trace)
    
    #下载EUImap的fits文件
    imaging_obstime = julian.from_jd(imaging_obstime_julian)
    filename = download_solo_eui_map(imaging_obstime, local_data_dir=local_data_dir)
    if filename:
        #读取FITS文件并创建SunPy Map
        eui_map = sunpy.map.Map(filename)
        rot_angle = eui_map.fits_header['CROTA']
        eui_map_rot = eui_map.rotate(rot_angle*u.deg)
        
        #计算磁力线足点和星下点在EUImap上的位置，arcsec。
        sub_solar_point_x, sub_solar_point_y = Carrington_To_Helioprojective(lon_beg, lat_beg, imaging_obstime, eui_map_rot.observer_coordinate)
        MFL_x, MFL_y = Carrington_To_Helioprojective(MFL_photosphere_lon_deg, MFL_photosphere_lat_deg, imaging_obstime, eui_map_rot.observer_coordinate)
        # 投影至卡林顿参考系
        shape_out = (720, 1440) 
        header = sunpy.map.make_fitswcs_header(shape_out,
                                               SkyCoord(180, 0, unit=u.deg,
                                                        frame="heliographic_carrington",
                                                        obstime=eui_map_rot.date,
                                                        observer=eui_map_rot.observer_coordinate ),
                                               scale=[360 / shape_out[1],
                                                      180 / shape_out[0]] * u.deg / u.pix,
                                               projection_code="CAR")

        eui_map_rot_in_carrington = eui_map_rot.reproject_to(header)
        
        #画图
        # ================ figure 1 ===================
        fig = plt.figure(figsize=(10,10))
        ax1 = fig.add_axes([0.225, 0.38, 0.55, 0.55], projection=eui_map_rot)
        
        xlims_world = [-3200, 3200]*u.arcsec
        ylims_world = [-3200, 3200]*u.arcsec
        world_coords = SkyCoord(Tx=xlims_world, Ty=ylims_world, frame=eui_map_rot.coordinate_frame)
        pixel_coords_x, pixel_coords_y = eui_map_rot.wcs.world_to_pixel(world_coords)

        eui_map_rot.plot(axes=ax1, clip_interval=(1, 99.9)*u.percent)

        coord = SkyCoord([sub_solar_point_x, MFL_x] * u.arcsec,
                         [sub_solar_point_y, MFL_y] * u.arcsec,
                         frame=eui_map_rot.coordinate_frame)
        ax1.plot_coord(coord, '-',color='red')
        coord = SkyCoord([sub_solar_point_x] * u.arcsec,
                         [sub_solar_point_y] * u.arcsec,
                         frame=eui_map_rot.coordinate_frame)
        ax1.plot_coord(coord, '*',markersize=8,color='blue',label='subsolar_point')
        coord = SkyCoord([MFL_x] * u.arcsec,
                         [MFL_y] * u.arcsec,
                         frame=eui_map_rot.coordinate_frame)
        ax1.plot_coord(coord, '+',markersize=8,color='blue',label='MFL_footpoint')
        ax1.legend()
        ax1.set_xlim(pixel_coords_x)
        ax1.set_ylim(pixel_coords_y)
        plt.text(0.01,0.95,'(a)',color='white', transform=ax1.transAxes)
        
        ax2 = fig.add_axes([0.1, 0.24, 0.8, 0.09])
        plt.plot(Epoch_orbit,Vr_vect,color='black')
        plt.plot([datetime_trace,datetime_trace],[ax2.get_ylim()[0],ax2.get_ylim()[1]],'--',color='red',linewidth=2)
        plt.text(0.01,0.85,'(b)', transform=ax2.transAxes)
        ax2.set_xticklabels([])
        plt.ylabel('Vr [km/s]')

        ax3 = fig.add_axes([0.1, 0.145, 0.8, 0.09])
        ax3.set_xticklabels([])
        plt.plot(Epoch_orbit,N_vect,color='black')
        plt.plot([datetime_trace,datetime_trace],[ax3.get_ylim()[0],ax3.get_ylim()[1]],'--',color='red',linewidth=2)
        plt.text(0.01,0.85,'(c)', transform=ax3.transAxes)
        plt.ylabel(r'N [$cm^{-3}$]')

        ax4 = fig.add_axes([0.1, 0.05, 0.8, 0.09])
        a = plt.plot(Epoch_orbit,T_vect,color='black')
        plt.plot([datetime_trace,datetime_trace],[ax4.get_ylim()[0],ax4.get_ylim()[1]],'--',color='red',linewidth=2)
        date_format = mpl.dates.DateFormatter('%d/%H:%M')
        plt.text(0.01,0.85,'(d)', transform=ax4.transAxes)
        plt.ylabel(r'T [eV]')
        ax4.set_xlabel('2023-03')
        ax4.xaxis.set_major_formatter(date_format)


        figure_name = 'SolO_eui_FSI_174_full_disk_'+eui_map_rot.fits_header['DATE-OBS'].replace(':','_').replace('.','_')+'.png'
        plt.savefig(fulldisk_figure_dir + figure_name, dpi=300)
        plt.text(0.01,1.05,'created by: plot_SolO_EUI_FSI_movie_and_traceback.ipynb', transform=ax1.transAxes)
        plt.close()

        # 画卡林顿图
        # ================ figure 2 ===================
        fig = plt.figure(figsize=(10,10))
        ax1 = fig.add_axes([0.1, 0.45, 0.8, 0.4], projection=eui_map_rot_in_carrington)
        eui_map_rot_in_carrington.plot(axes=ax1)
        MFL_photosphere_lon_deg, MFL_photosphere_lat_deg, lon_beg, lat_beg
        coord = SkyCoord([lon_beg, MFL_photosphere_lon_deg] * u.deg,
                         [lat_beg, MFL_photosphere_lat_deg] * u.deg,
                                 frame=eui_map_rot_in_carrington.coordinate_frame)
        ax1.plot_coord(coord, '-',color='red')
        coord = SkyCoord([lon_beg] * u.deg,
                         [lat_beg] * u.deg,
                         frame=eui_map_rot_in_carrington.coordinate_frame)
        ax1.plot_coord(coord, '*',markersize=8,color='blue',label='subsolar_point')
        coord = SkyCoord([MFL_photosphere_lon_deg] * u.deg,
                         [MFL_photosphere_lat_deg] * u.deg,
                         frame=eui_map_rot_in_carrington.coordinate_frame)
        ax1.plot_coord(coord, '+',markersize=8,color='blue',label='MFL_footpoint')
        ax1.legend()
        plt.text(0.01,0.95,'(a)',color='black', transform=ax1.transAxes)

        ax2 = fig.add_axes([0.1, 0.29, 0.8, 0.09])
        plt.plot(Epoch_orbit,Vr_vect,color='black')
        plt.plot([datetime_trace,datetime_trace],[ax2.get_ylim()[0],ax2.get_ylim()[1]],'--',color='red',linewidth=2)
        plt.text(0.01,0.85,'(b)', transform=ax2.transAxes)
        ax2.set_xticklabels([])
        plt.ylabel('Vr [km/s]')

        ax3 = fig.add_axes([0.1, 0.195, 0.8, 0.09])
        ax3.set_xticklabels([])
        plt.plot(Epoch_orbit,N_vect,color='black')
        plt.plot([datetime_trace,datetime_trace],[ax3.get_ylim()[0],ax3.get_ylim()[1]],'--',color='red',linewidth=2)
        plt.text(0.01,0.85,'(c)', transform=ax3.transAxes)
        plt.ylabel(r'N [$cm^{-3}$]')

        ax4 = fig.add_axes([0.1, 0.1, 0.8, 0.09])
        a = plt.plot(Epoch_orbit,T_vect,color='black')
        plt.plot([datetime_trace,datetime_trace],[ax4.get_ylim()[0],ax4.get_ylim()[1]],'--',color='red',linewidth=2)
        date_format = mpl.dates.DateFormatter('%d/%H:%M')
        plt.text(0.01,0.85,'(d)', transform=ax4.transAxes)
        plt.ylabel(r'T [eV]')
        ax4.set_xlabel('2023-03')
        ax4.xaxis.set_major_formatter(date_format)
        
        figure_name = 'SolO_eui_FSI_174_synoptic_'+eui_map_rot.fits_header['DATE-OBS'].replace(':','_').replace('.','_')+'.png'
        plt.savefig(synoptic_figure_dir + figure_name, dpi=300)
        plt.close()
# 程序结束